In [58]:
import sys
sys.path.append("..")
from master import cds_utils as utils
# from master.cds_utils import *
import pandas as pd
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.preprocessing import MinMaxScaler
import os

In [59]:
## Preparing Dataset

def preprocessing(dataframe_x,labels):
    list_of_x=[]
    list_of_y=np.zeros([1,1])

    for i in dataframe_x:
        list_of_x.append(np.append(0,i))
    
    list_of_y.fill(labels[2])
    
    # reshape input to be [samples, time steps, features]
    list_of_x = np.reshape(list_of_x, (1, 60, 17))
    
    return list_of_x,list_of_y

In [60]:
## Training and evaluating the model

## Train model in windows
def train_model(X_train,y_train):
        # fix random seed for reproducibility
        np.random.seed(7)
        
        model.fit(X_train, y_train, epochs=3, batch_size=60)
        
        
## Evaluates model based on test set
def evaluate_model(X_train,y_train,accuracy=0,iteration=0):
        
        # Final evaluation of the model
        scores = model.evaluate(X_train, y_train, verbose=0)
        
        #To get average accuracy of model
        accuracy+=scores[1]*100
        iteration+=1
        prediction=model.predict(X_train)
        
        return accuracy,iteration


In [61]:
if __name__=='__main__':

    # STEP 0: Check if combined dataset exists. If exists, GO TO STEP 4
    filename = '../master/AAPL - main.csv'
    if os.path.isfile(filename):
        dataset = pd.read_csv(filename, parse_dates=[0])
        dataset.set_index('Date', inplace=True)
    else:
        # STEP 1:: Aggregate financial data from discrete csv files
        #compile_financial('AAPL/')

        # STEP 2: Extract csv files into pandas dataframe
        # (1) financial dataframe
        financial = pd.read_csv('../master/AAPL - financial.csv', parse_dates=[1])
        financial = interpolate_data(financial, method='zero')
        # (2) price dataframe
        price = pd.read_csv('../master/AAPL - price.csv', parse_dates=[0]) # path to price 
        price.set_index('Date', inplace=True)
        # (3) technical dataframe
        technical = pd.read_csv('../master/AAPL - technical.csv', parse_dates=[0])
        technical.set_index('Date', inplace=True)
        # camel case column names for technical
        for old_column in technical:
            new_column = ' '.join([word.title() for word in old_column.split('_')])
            technical.rename(columns={old_column:new_column}, inplace=True)

        # STEP 3: Join different datasets based on overlapping dates
        dataset = combine_datasets(financial=financial, price=price, technical=technical)

    # STEP 4: Split the dataset into train and test
    train, test = utils.train_test_split(dataset, spl=0.4)

    # STEP 5: Parse the train/test dataframe into a data generator
    data_gen = utils.data_generator(train, train_days=60, next='day')
    i=0
   
    
    model = Sequential()
    model.add(LSTM(200))
    # each 60 outputs will have a dimension of 100
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    for x_train,y_train in data_gen:
        
        scaler = MinMaxScaler(feature_range=(0, 1))
        x_train=x_train.drop("Quarter",axis=1)
        x_train = scaler.fit_transform(x_train.values)
        x,y=preprocessing(x_train, y_train)
        
        train_model(x,y)
#         i+=1
#         if i >10:
#             break
            
    data_gen = utils.data_generator(test, train_days=60, next='day')
#     j=0
    accuracy=0
    iteration=0
    for x_test,y_test in data_gen:
        scaler = MinMaxScaler(feature_range=(0, 1))
        x_test=x_test.drop("Quarter",axis=1)
        x_test = scaler.fit_transform(x_test.values)
        x,y=preprocessing(x_test, y_test)
        accuracy,iteration=evaluate_model(x,y,accuracy,iteration)
        
#         j+=1
#         if j>10:
#             break
    print("Average")
    print(accuracy/iteration)
    



**********************************************************
Iteration 0
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 4s 4s/step - loss: 0.7055 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 52ms/step - loss: 0.6083 - acc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 51ms/step - loss: 0.4909 - acc: 1.0000
**********************************************************
Iteration 1
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 51ms/step - loss: 1.1786 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 53ms/step - loss: 1.1551 - acc: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 60ms/step - loss: 1.0731 - acc: 0.0000e+00
**********************************************************
Iteration 2
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 55ms/step - loss: 0.4798 - acc: 1.0000
Epoch 2/3


**********************************************************
Iteration 21
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 50ms/step - loss: 1.4203 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 50ms/step - loss: 1.3136 - acc: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 49ms/step - loss: 1.1674 - acc: 0.0000e+00
**********************************************************
Iteration 22
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 54ms/step - loss: 0.4545 - acc: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 52ms/step - loss: 0.4857 - acc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 51ms/step - loss: 0.5037 - acc: 1.0000
**********************************************************
Iteration 23
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 50ms/step - loss: 0.8793 - acc: 0.0000e+00
Epoch

1/1 [==============================] - 0s 52ms/step - loss: 0.5647 - acc: 1.0000
**********************************************************
Iteration 42
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 51ms/step - loss: 0.8289 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 51ms/step - loss: 0.8150 - acc: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 51ms/step - loss: 0.7985 - acc: 0.0000e+00
**********************************************************
Iteration 43
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 50ms/step - loss: 0.7811 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 50ms/step - loss: 0.7609 - acc: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 51ms/step - loss: 0.7393 - acc: 0.0000e+00
**********************************************************
Iteration 44
Generating (60, 17) rows of training data
Epoch 1

Epoch 3/3
1/1 [==============================] - 0s 53ms/step - loss: 0.7513 - acc: 0.0000e+00
**********************************************************
Iteration 63
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 46ms/step - loss: 0.7504 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 44ms/step - loss: 0.7457 - acc: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 43ms/step - loss: 0.7389 - acc: 0.0000e+00
**********************************************************
Iteration 64
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 42ms/step - loss: 0.7380 - acc: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 42ms/step - loss: 0.7274 - acc: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 46ms/step - loss: 0.7153 - acc: 0.0000e+00
**********************************************************
Iteration 65
Generating (60, 17) rows of trainin

Epoch 3/3
1/1 [==============================] - 0s 49ms/step - loss: 1.1800 - acc: 0.0000e+00
**********************************************************
Iteration 84
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 51ms/step - loss: 0.3919 - acc: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 48ms/step - loss: 0.4103 - acc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 49ms/step - loss: 0.4234 - acc: 1.0000
**********************************************************
Iteration 85
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==============================] - 0s 47ms/step - loss: 0.4341 - acc: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 52ms/step - loss: 0.4396 - acc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 54ms/step - loss: 0.4425 - acc: 1.0000
**********************************************************
Iteration 86
Generating (60, 17) rows of training data
Epoch 1/3
1/1 [==

...

**********************************************************
Iteration 1125
Generating (60, 17) rows of training data
**********************************************************
Iteration 1126
Generating (60, 17) rows of training data
**********************************************************
Iteration 1127
Generating (60, 17) rows of training data
**********************************************************
Iteration 1128
Generating (60, 17) rows of training data
**********************************************************
Iteration 1129
Generating (60, 17) rows of training data
**********************************************************
Iteration 1130
Generating (60, 17) rows of training data
**********************************************************
Iteration 1131
Generating (60, 17) rows of training data
**********************************************************
Iteration 1132
Generating (60, 17) rows of training data
**********************************************************
Iteration 113

**********************************************************
Iteration 1205
Generating (60, 17) rows of training data
**********************************************************
Iteration 1206
Generating (60, 17) rows of training data
**********************************************************
Iteration 1207
Generating (60, 17) rows of training data
**********************************************************
Iteration 1208
Generating (60, 17) rows of training data
**********************************************************
Iteration 1209
Generating (60, 17) rows of training data
**********************************************************
Iteration 1210
Generating (60, 17) rows of training data
**********************************************************
Iteration 1211
Generating (60, 17) rows of training data
**********************************************************
Iteration 1212
Generating (60, 17) rows of training data
**********************************************************
Iteration 121

**********************************************************
Iteration 1285
Generating (60, 17) rows of training data
**********************************************************
Iteration 1286
Generating (60, 17) rows of training data
**********************************************************
Iteration 1287
Generating (60, 17) rows of training data
**********************************************************
Iteration 1288
Generating (60, 17) rows of training data
**********************************************************
Iteration 1289
Generating (60, 17) rows of training data
**********************************************************
Iteration 1290
Generating (60, 17) rows of training data
**********************************************************
Iteration 1291
Generating (60, 17) rows of training data
**********************************************************
Iteration 1292
Generating (60, 17) rows of training data
**********************************************************
Iteration 129

**********************************************************
Iteration 1360
Generating (60, 17) rows of training data
**********************************************************
Iteration 1361
Generating (60, 17) rows of training data
**********************************************************
Iteration 1362
Generating (60, 17) rows of training data
**********************************************************
Iteration 1363
Generating (60, 17) rows of training data
**********************************************************
Iteration 1364
Generating (60, 17) rows of training data
**********************************************************
Iteration 1365
Generating (60, 17) rows of training data
**********************************************************
Iteration 1366
Generating (60, 17) rows of training data
**********************************************************
Iteration 1367
Generating (60, 17) rows of training data
**********************************************************
Iteration 136

**********************************************************
Iteration 1432
Generating (60, 17) rows of training data
**********************************************************
Iteration 1433
Generating (60, 17) rows of training data
**********************************************************
Iteration 1434
Generating (60, 17) rows of training data
**********************************************************
Iteration 1435
Generating (60, 17) rows of training data
**********************************************************
Iteration 1436
Generating (60, 17) rows of training data
**********************************************************
Iteration 1437
Generating (60, 17) rows of training data
**********************************************************
Iteration 1438
Generating (60, 17) rows of training data
**********************************************************
Iteration 1439
Generating (60, 17) rows of training data
**********************************************************
Iteration 144

# Findings

a/b ratio split where a refers to train set and b refers to test set

Accuracy of:

Average
52.84974093264249
On test data with 90/10 ratio split and 100 neurons in LSTM layer

Average
52.84974093264249
On test data with 90/10 ratio split and 200 neurons in LSTM layer

Average
53.960396039603964
On test data with 50/50 ratio split and 200 neurons in LSTM layer

Average
53.68349249658936
On test data with 40/60 ratio split and 200 neurons in LSTM layer

In [ ]:
## Movie Example

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))